# 셀레니움을 이용한 유튜브 크롤링

In [12]:
import time
import requests
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException 
from selenium.webdriver.common.keys import Keys #스크롤다운의 key

In [13]:
keyword = 'care routine'
# search = keyword.replace('_','+')
search = keyword.replace(' ','+')
# chrome webdriver
driver = webdriver.Chrome('C:/Users/snari/Downloads/chromedriver_win32/chromedriver.exe')
url = f"https://www.youtube.com/results?search_query={search}"
driver.get(url)

#페이지 로딩
# By.ID - 태그에 있는 ID 로 검색
# By.CSS_SELECTOR - CSS Selector 로 검색
# By.NAME - 태그에 있는 name 으로 검색
# By.TAG_NAME - 태그 이름으로 검색
try:
    myElem = WebDriverWait(driver, 5).until(EC.presence_of_element_located((By.ID, 'UD7Dzf')))
    print("크롤링 준비 완료")
except TimeoutException:
    print("페이지 로딩 시간 초과")
    exit()
except Exception as e:
    print(e)
    exit()

크롤링 준비 완료


In [58]:
#스크롤 다운 전 크롤링(requests 사용)
#URL의 초기페이지에 나와있는 게시글만 확인 가능함
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
found_items = soup.select('div.yt-lockup-content > h3 > a')
len(found_items)

20

In [15]:
#html body 영역
elem = driver.find_element_by_tag_name("body")

In [16]:
l=0
num_of_pagedowns = 10
while num_of_pagedowns: #0일 때 while 종료
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.5)
    num_of_pagedowns -= 1
    l += 1
    print(l)

1
2
3
4
5
6
7
8
9
10


In [20]:
#마지막 페이지 체크 함수
#'결과더이상없습니다'(message)가 나올때까지 message 나오면 TRUE 리턴
def check_end():
    try:
        message = driver.find_element_by_id('message')
    except NoSuchElementException:
        return False
    else: return True

In [21]:
print("스크롤 다운 시작")
l = 0
while not check_end(): #check_end가 TRUE가 리턴될때까지 실행
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.1)
    l += 1
    if l % 5 == 0:
        print(l, "번째 스크롤 다운")
    # 6번째서 종료
#     if l == 6:
#         break
print("스크롤 다운 종료")


스크롤 다운 시작
스크롤 다운 종료


In [17]:
#스크롤 다운 후 크롤링
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
found_items = soup.select('div.style-scope > ytd-video-renderer.style-scope')
len(found_items)

58

In [19]:
data = []
i=0
for item in found_items:
    i += 1
    try:
        thumbnail = item.select_one('a.yt-simple-endpoint > yt-img-shadow.ytd-thumbnail> img.yt-img-shadow')['src']
    except:
#         thumbnail = item.select_one('img', {'id' : 'img'})['src']
        thumbnail = "None"
    title = item.select_one('h3.title-and-badge > a').text
    raw_viewcount = item.select_one('div.ytd-video-meta-block > span').text
#     raw_viewcount = item.select_one('div.style-scope_ytd-video-meta-block > span').text     style-scope만쓰면 안됨
    viewcount = raw_viewcount[3:]
#     a = item.select_one('div.ytd-video-meta-block > div.style-scope')
#     a = item.find('div', {'id' : 'ex_id'})
    name = item.select_one('yt-formatted-string.style-scope > a').text
    length = item.select_one('ytd-thumbnail-overlay-time-status-renderer.style-scope > span').text
    raw_url = item.select_one('a',{'id':'thumbnail'})['href']
    url = 'https://www.youtube.com'+ raw_url
#     length = item.find_element_by_class_name('style-scope ytd-thumbnail-overlay-time-status-renderer').get_attribute("aria-label")
    value = [thumbnail, title, viewcount, name, length, url]
    data.append(value)
    
    


                self care routine (pamper day)
                48만회 Gabriella Whited

                Pony's Nighttime Skin Care Routine 포니 스킨케어 루틴 | Go To Bed With Me | Harper's BAZAAR
                404만회 Harper's BAZAAR

                MY LIP CARE ROUTINE!
                247만회 Luhhsetty

                MY BODY CARE ROUTINE | Glowy Skin, What I Eat, Stretch Marks, Hair Removal… | Annie Jaffrey
                32만회 AnnieJaffrey

                MY -Pamper- SELF CARE ROUTINE
                84만회 AlexandrasGirlyTalk

                Body Care Routine | Dry Brushing For Body and Face | Model Tips
                29만회 Valeria Lipovetsky

                self-care day! (my pamper routine)
                38만회 Sadie Aldis

                Self Care Routine | mind.body.soul
                7만회 Renee Amberg

                MY HAIR CARE ROUTINE 2018
                89만회 Amy Macedo

                My Self-Care Routine | How to Pamper, Reset, Recharge & De-stress
                6.8만회 Dan

In [30]:
df = pd.DataFrame(data, columns=['썸네일', '제목', '조회수', '닉네임', '재생시간', 'URL'])

In [36]:
df.to_excel('C:/Users/snari/python study/2주차_new/youtube.xlsx', index=False)

In [4]:
found_box = driver.find_elements_by_css_selector("#contents > ytd-video-renderer")
len(found_box)

20

In [9]:
for box in found_box:
    title = box.find_element_by_css_selector('#video-title')
    link = title.get_attribute("href")
    print(link)

https://www.youtube.com/watch?v=dM1vWwidK9g
https://www.youtube.com/watch?v=Cz_-SD_6SdE
https://www.youtube.com/watch?v=fgELyoHZSWg
https://www.youtube.com/watch?v=eiqeSMyDNkQ
https://www.youtube.com/watch?v=8rawD5i2-Lo
https://www.youtube.com/watch?v=unaRagDUS30
https://www.youtube.com/watch?v=uz50DSdXb8k
https://www.youtube.com/watch?v=hvTmYvCChGc
https://www.youtube.com/watch?v=dakFbtpH7hw
https://www.youtube.com/watch?v=2aUL2Jzdnoc
https://www.youtube.com/watch?v=R28RNkRGJKs
https://www.youtube.com/watch?v=qANGyUxtTQM
https://www.youtube.com/watch?v=yDela477waA
https://www.youtube.com/watch?v=sGE1AGd9UV8
https://www.youtube.com/watch?v=7vfxRU4Id0g
https://www.youtube.com/watch?v=dUo388-duuU
https://www.youtube.com/watch?v=9NYvhm0cRx8
https://www.youtube.com/watch?v=OZYgGUvUf0E
https://www.youtube.com/watch?v=xo-F2dttJe0
https://www.youtube.com/watch?v=T_4DhyYzHr8
